In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time


# define a function to get information from website
def get_url_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    try:
        url_info = requests.get(url,headers=headers)
        url_soup = BeautifulSoup(url_info.text, 'html.parser')
        return url_soup
    except:
        print('sorry')


# information list
names = [] 
diractors = []
script_writers = []
actors = []
scores = []
types = []
countries = []
votes = []

# get the Douban top 250 films information
for i in range(10):
    film_page_url = 'https://movie.douban.com/top250?start={}&filter='.format(i*25) # the url of every page
    time.sleep(random.randint(1,3)) 
    page_info = get_url_info(film_page_url)

    # get the url of every film specific introduction
    
    film_selected_info = page_info.find_all(attrs={'class':'hd'})
    film_list = [info.a['href'] for info in film_selected_info]

    # get the specific information
    for film in film_list:
        time.sleep(random.randint(1,3))
        specific_film_info = get_url_info(film)

        names.append(specific_film_info.find(attrs={'property':'v:itemreviewed'}).text.split(' ')[0])
        diractors.append(specific_film_info.find(attrs={'id':'info'}).text.split('\n')[1:11][0].split(': ')[1])

        script_writers.append(specific_film_info.find(attrs={'id':'info'}).text.split('\n')[1:11][1].split(': ')[1].split('/')[0])
        actors.append(specific_film_info.find(attrs={'id':'info'}).text.split('\n')[1:11][2].split(': ')[1].split('/')[0])

        types.append(specific_film_info.find(attrs={'id':'info'}).text.split('\n')[1:11][3].split(': ')[1])
        countries.append(specific_film_info.find(attrs={'id':'info'}).text.split('\n')[1:11][4].split(': ')[1])

        scores.append(specific_film_info.find(attrs={'property': 'v:average'}).text)
        votes.append(specific_film_info.find(attrs={'property': 'v:votes'}).text)

raw_data = {'name':names, 'diractor':diractors, 'scriptwriter':script_writers, 'actor':actors, 'country':countries, 'score':scores, 'vote':votes, 'type':types}
data = pd.DataFrame(raw_data)
print(data)

              name  diractor scriptwriter        actor  \
0           肖申克的救赎  弗兰克·德拉邦特    弗兰克·德拉邦特       蒂姆·罗宾斯    
1             霸王别姬       陈凯歌          芦苇          张国荣    
2             阿甘正传  罗伯特·泽米吉斯      艾瑞克·罗斯       汤姆·汉克斯    
3            泰坦尼克号   詹姆斯·卡梅隆      詹姆斯·卡梅隆  莱昂纳多·迪卡普里奥    
4          这个杀手不太冷     吕克·贝松        吕克·贝松        让·雷诺    
..             ...       ...          ...          ...   
245            我爱你       秋昌旼          姜草          宋在浩    
246          完美陌生人   保罗·杰诺维塞     保罗·杰诺维塞       马可·贾利尼    
247         花束般的恋爱      土井裕泰         坂元裕二        菅田将晖    
248             弱点  约翰·李·汉考克    约翰·李·汉考克      桑德拉·布洛克    
249  哈利·波特与死亡圣器(上)     大卫·叶茨    史蒂夫·克洛夫斯    丹尼尔·雷德克里夫    

                     country score     vote               type  
0                         美国   9.7  2683748            剧情 / 犯罪  
1                中国大陆 / 中国香港   9.6  1990775       剧情 / 爱情 / 同性  
2                         美国   9.5  2015476            剧情 / 爱情  
3      美国 / 墨西哥 / 澳大利亚 / 加拿大   9.4  1974520

In [17]:
data.to_csv('/home/bio_kang/data/top250_films.csv', index=None, encoding='gbk')

essential step

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import re


# the function is designed to get url information
def get_url_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    try:
        time.sleep(random.randint(1,2))
        info = requests.get(url, headers=headers).text
        soup = BeautifulSoup(info, 'html.parser')
        return soup
    except:
        print('Sorry! The film information is not got')


download page information (just run one time) 

In [71]:
for i in range(10):
    time.sleep(random.randint(1,3))
    page_info = get_url_info('https://movie.douban.com/top250?start={}&filter='.format(i*25))
    with open('/home/bio_kang/Learning/Python/film_project/film_page_{}.html'.format(i), 'w') as f:
        f.write(str(page_info))

get all film urls, the number of people who voted and the score of all films of all pages

In [15]:
# information
urls = [] # each film url
votes = [] # the vote people number
scores = [] # the score of film
names = [] # film name


for i in range(10):
    with open('/home/bio_kang/Learning/Python/film_project/film_page_{}.html'.format(i), 'r') as f:
        page_info = f.read()

        pattern_url = re.compile(r'<a class="" href="(.*?)">')
        pattern_vote = re.compile(r'<span>(\d+.*?)</span>')
        pattern_score = re.compile(r'<span class="rating_num" property="v:average">(.*?)</span>')
        pattern_name = re.compile('<span class="title">([^\s].*?)</span>')

        urls = urls + re.findall(pattern_url, page_info)
        votes = votes + re.findall(pattern_vote, page_info)
        scores = scores + re.findall(pattern_score, page_info)
        names = names + re.findall(pattern_name, page_info)


In [16]:
def check(url):
    with open('/home/bio_kang/Learning/Python/film_project/download.csv','r') as f: # overlop write
        downloads = f.read().strip('\n').split('\n')
        if url in downloads:
            return "False"
        else:
            with open('/home/bio_kang/Learning/Python/film_project/download.csv', 'r+') as k:
                url_add = k.read().strip('\n').split('\n')
                url_add = list(set(downloads))
                url_add.append(url)
            with open('/home/bio_kang/Learning/Python/film_project/download.csv', 'w') as j:
                j.write("\n".join(url_add))
            return "True" 


process data

In [17]:
new_votes = []
for vote in votes:
    pattern = re.compile(r'\d+')
    new_votes.append(str(re.findall(pattern, vote)).strip('[]').strip('\''))

print(new_votes)

['2684629', '1991412', '2016057', '1975111', '2159805', '1237725', '2092225', '1032649', '1929049', '1631278', '1316345', '1550546', '1571020', '1747383', '1235221', '1218707', '1249185', '1773039', '1433204', '872348', '485195', '893326', '1422830', '1008292', '1709788', '1182242', '802916', '1531882', '986496', '773383', '1080010', '746318', '643370', '634870', '1237130', '1959975', '1469888', '968240', '447228', '941232', '1270025', '579614', '920876', '1142018', '1569871', '767003', '570243', '697324', '814428', '521088', '641568', '894256', '783342', '787228', '397946', '659790', '507013', '1510838', '553537', '776846', '791697', '570089', '717122', '919409', '525928', '513237', '1012552', '558681', '910582', '1163236', '559262', '603714', '517346', '380203', '1291468', '761114', '818908', '800284', '900329', '854770', '341216', '872318', '385105', '659519', '774981', '586595', '276897', '898895', '888831', '1015143', '651582', '758337', '522674', '651352', '802922', '418486', '66

downloads and check function

In [18]:
# get everyone film information of html
for url in urls:
    if check(url) == 'True':
        time.sleep(random.randint(1,3))
        print('the program have started downloading {} film'.format(urls.index(url)))
        film_info = get_url_info(url)
        with open('/home/bio_kang/Learning/Python/film_project/film_info/film_{}.html'.format(urls.index(url)), 'w') as f:
            f.write(str(film_info))
            print('the program finish downloading {} film'.format(urls.index(url)))
    else:
        continue

the program have started downloading 0 film
the program finish downloading 0 film
the program have started downloading 1 film
the program finish downloading 1 film
the program have started downloading 2 film
the program finish downloading 2 film
the program have started downloading 3 film
the program finish downloading 3 film
the program have started downloading 4 film
the program finish downloading 4 film
the program have started downloading 5 film
the program finish downloading 5 film
the program have started downloading 6 film
the program finish downloading 6 film
the program have started downloading 7 film
the program finish downloading 7 film
the program have started downloading 8 film
the program finish downloading 8 film
the program have started downloading 9 film
the program finish downloading 9 film
the program have started downloading 10 film
the program finish downloading 10 film
the program have started downloading 11 film
the program finish downloading 11 film
the program 

unique number

In [75]:
uni_num = []
years = []
countries = []
directors = []
actors = []
descriptions = []
for i in range(250):
    with open('/home/bio_kang/Learning/Python/film_project/film_info/film_{}.html'.format(i), 'rb') as f:
        film_info = f.read().decode('utf-8')

        pattern_uni_num = re.compile(r'<span class="pl">IMDb:</span> (.*?)<br/>') # unique number
        pattern_year = re.compile(r'<span class="year">\((.*?)\)</span>') # year
        pattern_country = re.compile(r'<span class="pl">制片国家/地区:</span>(.*?)<br/>') # country
        pattern_director = re.compile(r'<meta content=(.*?) property="video:director"/>') # director
        pattern_actor = re.compile(r'<meta content="(.*?)" property="video:actor"/>') # actors
        pattern_description = re.compile(r'<meta content="(.*?)property="og:description">') # description

        uni_num.append(str(re.findall(pattern_uni_num, film_info)).strip("[]").strip("'"))
        years.append(str(re.findall(pattern_year, film_info)).strip("[]").strip("'"))
        countries.append(str(re.findall(pattern_country, film_info)).strip("[]").strip("'").split('/')[0])
        directors.append(re.findall(pattern_director, film_info))
        actors.append(re.findall(pattern_actor, film_info))
        descriptions.append(str(re.findall(pattern_description, film_info)).strip('[]').strip('\''))


process directors data

In [76]:
new_director = []
for director in directors:
    if len(director) > 1:
        new_director.append(str(director[0]).strip('[]').strip('\'').strip('\"'))
    else:
        new_director.append(str(director).strip('[]').strip('\'').strip('\"'))


process actor data

In [77]:
new_actor = []
for i in range(250):
    if len(actors[i]) > 5:
        new_actor.append(str(actors[i][:5]).strip('[]').strip('\"'))
    else:
        new_actor.append(str(actors[i]).strip('[]').strip('\"'))


In [78]:
raw_data = {'encoding':uni_num, 'name':names, 'description':descriptions, 'country':countries, 'director':new_director, 'actor':new_actor, 'vote':new_votes, 'score':scores, 'year':years, 'link':urls }
data = pd.DataFrame(raw_data)
data.to_csv('/home/bio_kang/Learning/Python/film_project/top250_film_info.csv', index=None, encoding='gbk')

In [12]:
import pandas as pd
data = pd.read_csv('/home/bio_kang/Learning/Python/film_project/top250_film_info.csv', 'r', encoding='gbk')

